In [1]:
import os
os.environ["PYSPARK_PYTHON"]="python3.7"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3.7"

In [2]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
            .master("spark://spark-master:7077") \
            .appName("Project") \
            .getOrCreate()

In [3]:
static = spark.read.format("json").load("hdfs://namenode/user/root/input/data10.json", multiLine = "true")

In [4]:
dataSchema = static.schema

In [5]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
.json("hdfs://namenode/user/root/input/data10.json")

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [9]:
category = streaming.groupBy("category").count()

In [10]:
query = category.writeStream.queryName("spark")\
.format("memory").outputMode("complete")\
.start()

In [12]:
spark.sql('SELECT * FROM spark').show()

+--------+-----+
|category|count|
+--------+-----+
+--------+-----+



In [5]:
df = df.na.drop().dropDuplicates()

In [6]:
df = df.filter(size(df['comments']) >= 30)

In [7]:
df.createOrReplaceTempView("dfTable")

In [8]:
!pip install textblob

     |████████████████████████████████| 636 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 16.5 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 301 kB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 719 kB 8.1 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 7.2 MB/s  eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=dc34d254838c64535b206f1d6175d28f0372194a1f77cc950da072775f1a96f5
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


In [9]:
from textblob import TextBlob

def to_sentiment(comments):
    positive, negative, neutral = 0, 0, 0
    for comment in comments:
        sentiment = TextBlob(comment).polarity
        if sentiment > 0:
            positive += 1
        elif sentiment < 0:
            negative += 1
        else:
            neutral += 1
            
    return [positive, negative, neutral]

convert = udf(to_sentiment, ArrayType(IntegerType()))

In [10]:
df = df.withColumn("comments", convert(col("comments")))

In [11]:
df = df.withColumn('positive', df.comments[0]) \
        .withColumn('negative', df.comments[1]) \
        .withColumn('neutral', df.comments[2])

In [12]:
df = df.drop('comments')

In [14]:
df.printSchema()

root
 |-- androidVersion: string (nullable = true)
 |-- category: string (nullable = true)
 |-- contentRating: string (nullable = true)
 |-- currentVersion: string (nullable = true)
 |-- installs: long (nullable = true)
 |-- lastUpdate: long (nullable = true)
 |-- price: double (nullable = true)
 |-- ratings: long (nullable = true)
 |-- reviews: long (nullable = true)
 |-- score: double (nullable = true)
 |-- size: string (nullable = true)
 |-- title: string (nullable = true)
 |-- positive: integer (nullable = true)
 |-- negative: integer (nullable = true)
 |-- neutral: integer (nullable = true)



In [17]:
df.select('neutral').show()

+-------+
|neutral|
+-------+
|      0|
|     13|
|     15|
|     35|
|     34|
|     12|
|     16|
|      3|
|      6|
|      0|
|      8|
|     24|
|      6|
|     35|
|     19|
|     37|
|      9|
|     10|
|      2|
|      8|
+-------+
only showing top 20 rows



In [ ]:
import re

def remove_character(size):
    """
        This UDF takes size as input and returns number of size
    """
    return re.sub(r'[^\d]', '', size)

convert_to_number = udf(remove_character, IntegerType())

In [ ]:
df = df.withColumn("size", convert_to_number(col("size")))
df.show(2)

In [ ]:
df.select('size').collect()